In [1]:
import random
import sys


def randomInteger(limit):
    return random.randint(-limit, limit)


def selecionar(cantidad=8):
    seleccion = []
    for _ in range(cantidad):
        seleccion.append([#xyzwv
            randomInteger(50), randomInteger(50),
            randomInteger(50), randomInteger(50), randomInteger(50)])
    return seleccion


poblacion = selecionar()


Generamos una poblacion de ejemplo 

In [2]:
print(poblacion)

[[27, 19, 20, -45, 4], [-10, 26, -46, -30, 48], [-19, 19, 16, 15, 1], [5, -33, 6, -10, -28], [7, 45, -1, 21, -25], [4, -40, -28, -32, -8], [-4, -50, -28, -3, 23], [33, -5, -38, -9, 13]]


In [3]:
import math


def funcionAptitud(invividuo: list) -> int:
    x=invividuo[0]
    y=invividuo[1]
    z=invividuo[2]
    w=invividuo[3]
    v=invividuo[4]
    fx = ( x + 2 * y + 3 * z + 4 *w + 5 * v)
    ## El minimo es 0 
    ## Cuando la suma da 50
    return  abs ( fx - 50) 

#Si todos son 0 la aptitud vale 50
funcionAptitud([0,0,0,0,0])

50

In [4]:
for invividuo in poblacion:
    print( "Invividuo" , invividuo  , " aptitud" , funcionAptitud(invividuo))

Invividuo [27, 19, 20, -45, 4]  aptitud 85
Invividuo [-10, 26, -46, -30, 48]  aptitud 26
Invividuo [-19, 19, 16, 15, 1]  aptitud 82
Invividuo [5, -33, 6, -10, -28]  aptitud 273
Invividuo [7, 45, -1, 21, -25]  aptitud 3
Invividuo [4, -40, -28, -32, -8]  aptitud 378
Invividuo [-4, -50, -28, -3, 23]  aptitud 135
Invividuo [33, -5, -38, -9, 13]  aptitud 112


In [5]:
def reproduccionPar(invividuoA: list, invividuoB: list) -> tuple:
    point = random.randint(1, len(invividuoA) - 2)
    new1 = invividuoA[:point] + invividuoB[point:]
    
    # ratio de 100% siempre se reproduce
    return new1


In [6]:
print(reproduccionPar([0,0,0,0], [1,1,1,1]))


[0, 0, 1, 1]


In [7]:

import itertools 
def cruzamiento(poblacionInicio :list , cantidadDeHijos= 16):
    # genera cantidad de hijos apartir de pob inicial

    #Genera todos los pares
    # 1 ,2,3 -> (1,2),(2,3),(1,3)
    todosLosPares= list(itertools.combinations(poblacionInicio , 2) )

    #Ordenamiento aleatorio
    # (1,2),(2,3),(1,3) -> (2,3),(1,3),(1,2) 
    random.shuffle(todosLosPares)
    
    #[:solo hasta la cantidad deseada]
    seleccionados =  todosLosPares [:cantidadDeHijos]

    hijos  = [ reproduccionPar(parSelecto[0],parSelecto[1]) for parSelecto in seleccionados ]
    return hijos


cruzamiento(poblacion, cantidadDeHijos=3)






[[-19, 19, -28, -3, 23], [-19, 19, 16, -9, 13], [-10, 45, -1, 21, -25]]

In [8]:
def mutarIndividuo(individuo:list , punto = -1):
    # Seleccion de elemento a cambiar
    punto = random.randint(0, len(individuo)-1) 
    #  primera parte + elemeto mutado + resto
    mutated = individuo[:punto] +[individuo[punto] + randomInteger(50)] + individuo[punto+1:]
    return mutated

def mutarPoblacion( poblacion , ratioDeMutacion = 0.1):
    poblacionMutada= []
    for invividuo in poblacion:
        # solo un porcentaje% se muta
        if (random.random() < ratioDeMutacion):
            poblacionMutada.append(mutarIndividuo(invividuo))
        else:
            poblacionMutada.append(invividuo)
    return poblacionMutada


In [9]:

def probailidadDeSeleccion( nuevaPoblacion : list):
    aptitudes = [funcionAptitud(invividuo) for invividuo in nuevaPoblacion ]
    sumaAptitudes = sum (aptitudes)
    probabilidades = [ aptitudDeIndividuo/sumaAptitudes for   aptitudDeIndividuo in aptitudes ]
    return probabilidades


def selecionarSiguientes(nuevaPoblacion: list, probabilidades: list, cantidadDeSeleccionados: int):
    parIndividuoProbabilidad = [{"indv":indv , "prob":prob} for indv , prob in zip (nuevaPoblacion, probabilidades)]
    
    # ordenamiento aleatorio con peso
    # menor probabilidad tiende al comienzo y mayor tiende al final
    parIndividuoProbabilidad.sort(key = lambda par:  random.random() * par["prob"] , reverse=True)
    
    #print("parIndividuoProbabilidad" , parIndividuoProbabilidad)

    selecionados =[indvSelecto["indv"] for indvSelecto in parIndividuoProbabilidad[:cantidadDeSeleccionados]]

    return selecionados


In [20]:
## Algoritmo Canonico

def Optimizar ( iteraciones):
    poblacion= selecionar(cantidad=8)
    max_fit = 1000
    
    for it in range(iteraciones):

        nuevaPoblacion = cruzamiento(poblacion, cantidadDeHijos= 8)
        nuevaPoblacionMutada = mutarPoblacion(nuevaPoblacion , ratioDeMutacion=0.1)
        probabilidadDeContinuar = probailidadDeSeleccion(nuevaPoblacionMutada)
        poblacion = selecionarSiguientes(nuevaPoblacionMutada , probabilidadDeContinuar, cantidadDeSeleccionados=8)
        
        
        #Mostar informacion -----------------------------------------------------

        max_fit = min(poblacion, key=lambda individ : funcionAptitud(individ))
        print( "it " ,it, " Max fit:",max_fit , "fitnss:" , funcionAptitud(max_fit))
        if  funcionAptitud(max_fit) == 0:break


    return sorted(poblacion , key =lambda individ : funcionAptitud(individ) )[0]



In [21]:
optimo = Optimizar(10000)

it  0  Max fit: [-7, 28, 43, -13, -17] fitnss: 9
it  1  Max fit: [-7, 72, 14, -14, -14] fitnss: 3
it  2  Max fit: [-7, 72, 14, -13, -17] fitnss: 8
it  3  Max fit: [-33, -37, -18, -5, 43] fitnss: 16
it  4  Max fit: [-33, 28, -18, -5, 43] fitnss: 114
it  5  Max fit: [-33, 28, 43, -13, -17] fitnss: 35
it  6  Max fit: [-33, 28, 61, -13, -17] fitnss: 19
it  7  Max fit: [-33, -37, 26, -5, 43] fitnss: 116
it  8  Max fit: [-33, 28, 26, -13, -17] fitnss: 86
it  9  Max fit: [-33, -37, -32, -5, 43] fitnss: 58
it  10  Max fit: [-53, -37, -32, -5, 43] fitnss: 78
it  11  Max fit: [-53, 28, -18, -5, 43] fitnss: 94
it  12  Max fit: [-53, 28, -18, -5, 43] fitnss: 94
it  13  Max fit: [-95, 28, -18, -5, 43] fitnss: 52
it  14  Max fit: [-53, 28, -18, -5, 43] fitnss: 94
it  15  Max fit: [-53, -37, -18, -5, 43] fitnss: 36
it  16  Max fit: [-53, -37, -18, -5, 43] fitnss: 36
it  17  Max fit: [-53, -37, -18, -5, 43] fitnss: 36
it  18  Max fit: [-13, -37, -18, -5, 43] fitnss: 4
it  19  Max fit: [-53, -37, -18, 

In [22]:
def printList(listI, title ):
    print(title , "-----------------")
    for ind in listI:

        print("\t" , ind)


def PruebaDeEscritorioOptimizar ( iteraciones):
    poblacion= selecionar(cantidad=8)
    max_fit = 1000
    
    for it in range(iteraciones):

        
        nuevaPoblacion = cruzamiento(poblacion, cantidadDeHijos= 8)
        printList(nuevaPoblacion,"cruzamiento")
        nuevaPoblacionMutada = mutarPoblacion(nuevaPoblacion , ratioDeMutacion=0.1)
        printList(nuevaPoblacionMutada,"mutarPoblacion")
        probabilidadDeContinuar = probailidadDeSeleccion(nuevaPoblacionMutada)
        printList(probabilidadDeContinuar,"probailidadDeSeleccion")
        poblacion = selecionarSiguientes(nuevaPoblacionMutada , probabilidadDeContinuar, cantidadDeSeleccionados=8)
        printList(poblacion,"selecionarSiguientes")
        
        #Mostar informacion -----------------------------------------------------

        max_fit = min(poblacion, key=lambda individ : funcionAptitud(individ))
        print( "it " ,it, " Max fit:",max_fit , "fitnss:" , funcionAptitud(max_fit))
        if  funcionAptitud(max_fit) == 0:break


    return sorted(poblacion , key =lambda individ : funcionAptitud(individ) )[0]

In [23]:
optimo = PruebaDeEscritorioOptimizar(2)

cruzamiento -----------------
	 [23, 19, -49, 2, -17]
	 [-23, 48, 39, -29, 35]
	 [40, -1, 39, -29, 35]
	 [-23, 40, 41, -17, 42]
	 [27, -19, 27, 33, 41]
	 [-23, 19, -49, 2, -17]
	 [23, 38, -20, -17, 42]
	 [-23, 27, -31, -39, -4]
mutarPoblacion -----------------
	 [23, 19, -49, 2, -17]
	 [-23, 48, 39, -29, 35]
	 [40, -1, 39, -29, 59]
	 [-23, 40, 41, -17, 42]
	 [27, -19, 27, 33, 41]
	 [-23, 19, -49, 2, -17]
	 [23, 38, -20, -17, 42]
	 [-23, 27, -31, -39, -4]
probailidadDeSeleccion -----------------
	 0.10634048926610085
	 0.09935097353969047
	 0.1417873190214678
	 0.13579630554168748
	 0.1782326510234648
	 0.1293060409385921
	 0.0654018971542686
	 0.14378432351472792
selecionarSiguientes -----------------
	 [40, -1, 39, -29, 59]
	 [-23, 19, -49, 2, -17]
	 [-23, 27, -31, -39, -4]
	 [-23, 40, 41, -17, 42]
	 [23, 19, -49, 2, -17]
	 [27, -19, 27, 33, 41]
	 [-23, 48, 39, -29, 35]
	 [23, 38, -20, -17, 42]
it  0  Max fit: [23, 38, -20, -17, 42] fitnss: 131
cruzamiento -----------------
	 [-23, 19